In [44]:
import pandas as pd

In [45]:
df = pd.read_csv('human.csv', encoding='cp949')
df.head()

,아이디,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,성별,자본 이득,자본 손실,주당 시간,모국
0,H20001,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,H20002,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,H20003,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,H20004,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,H20005,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [46]:
mdf = df.copy()

In [47]:
mdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   아이디     32561 non-null  object
 1   나이      32561 non-null  int64 
 2   노동 계급   30725 non-null  object
 3   fnlwgt  32561 non-null  int64 
 4   학력      32561 non-null  object
 5   교육 수    32561 non-null  int64 
 6   혼인 상태   32561 non-null  object
 7   직업      30718 non-null  object
 8   관계      32561 non-null  object
 9   인종      32561 non-null  object
 10  성별      32561 non-null  object
 11  자본 이득   32561 non-null  int64 
 12  자본 손실   32561 non-null  int64 
 13  주당 시간   32561 non-null  int64 
 14  모국      31978 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [48]:
mdf.isnull().sum()

아이디          0
나이           0
노동 계급     1836
fnlwgt       0
학력           0
교육 수         0
혼인 상태        0
직업        1843
관계           0
인종           0
성별           0
자본 이득        0
자본 손실        0
주당 시간        0
모국         583
dtype: int64

In [49]:
# 결측치 처리
mdf = mdf.dropna(subset=['노동 계급', '직업', '모국'])
mdf.shape

(30162, 15)

In [50]:
obj = ['노동 계급', '학력', '혼인 상태', '직업', '관계', '인종', '성별', '모국']
mdf[obj] = mdf[obj].apply(lambda x: x.astype('category').cat.codes)
mdf.head()

,아이디,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,성별,자본 이득,자본 손실,주당 시간,모국
0,H20001,39,5,77516,9,13,4,0,1,4,1,2174,0,40,38
1,H20002,50,4,83311,9,13,2,3,0,4,1,0,0,13,38
2,H20003,38,2,215646,11,9,0,5,1,4,1,0,0,40,38
3,H20004,53,2,234721,1,7,2,5,0,2,1,0,0,40,38
4,H20005,28,2,338409,9,13,2,9,5,2,0,0,0,40,4


In [51]:
from sklearn.model_selection import train_test_split  
X = mdf.drop(['아이디','성별'], axis=1) 
Y = mdf['성별']                    
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

In [52]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=6, random_state=0)

In [53]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, random_state=0)

In [54]:
display(tree.score(X_test, y_test))

0.8318525394510012

In [55]:
import xgboost as xgb
from xgboost import plot_importance
#import pan
import numpy as np

In [56]:
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

In [57]:
params = {'max_depth':3,
          'eta':0.1,
          'objective':'binary:logistic',
          'eval_metric':'logloss',
          'early_stoppings':100
         }
num_rounds = 400

In [58]:
# train 데이터 세트는 'train', evaluation(test) 데이터 세트는 'eval'로 명기
wlist = [(dtrain, 'train'),(dtest, 'eval')]
# 하이퍼 파라미터와 early stoppinig 파라미터를 train() 함수의 파라미터로 전달
xgb_model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist)

[10:53:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { early_stoppings } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.64407	eval-logloss:0.64343
[1]	train-logloss:0.60392	eval-logloss:0.60275
[2]	train-logloss:0.57052	eval-logloss:0.56893
[3]	train-logloss:0.54238	eval-logloss:0.54045
[4]	train-logloss:0.51846	eval-logloss:0.51623
[5]	train-logloss:0.49796	eval-logloss:0.49548
[6]	train-logloss:0.48012	eval-logloss:0.47721
[7]	train-logloss:0.46453	eval-logloss:0.46123
[8]	train-logloss:0.45088	eval-logloss:0.44734
[9]	train-logloss:0.43884	eval-logloss:0.43489
[10]	train-logloss:0.42637	eval-logloss:0.42244
[11]	train-logloss:0.41556	eval-logloss:0.41164
[12]	train-logloss:0.40673	eval-logloss:0

[162]	train-logloss:0.27976	eval-logloss:0.28319
[163]	train-logloss:0.27955	eval-logloss:0.28304
[164]	train-logloss:0.27935	eval-logloss:0.28295
[165]	train-logloss:0.27921	eval-logloss:0.28287
[166]	train-logloss:0.27909	eval-logloss:0.28294
[167]	train-logloss:0.27899	eval-logloss:0.28298
[168]	train-logloss:0.27896	eval-logloss:0.28297
[169]	train-logloss:0.27888	eval-logloss:0.28295
[170]	train-logloss:0.27884	eval-logloss:0.28299
[171]	train-logloss:0.27852	eval-logloss:0.28272
[172]	train-logloss:0.27838	eval-logloss:0.28274
[173]	train-logloss:0.27824	eval-logloss:0.28260
[174]	train-logloss:0.27801	eval-logloss:0.28264
[175]	train-logloss:0.27786	eval-logloss:0.28259
[176]	train-logloss:0.27783	eval-logloss:0.28260
[177]	train-logloss:0.27780	eval-logloss:0.28262
[178]	train-logloss:0.27766	eval-logloss:0.28262
[179]	train-logloss:0.27752	eval-logloss:0.28253
[180]	train-logloss:0.27722	eval-logloss:0.28228
[181]	train-logloss:0.27713	eval-logloss:0.28223
[182]	train-logloss:

[330]	train-logloss:0.26463	eval-logloss:0.28013
[331]	train-logloss:0.26454	eval-logloss:0.28006
[332]	train-logloss:0.26452	eval-logloss:0.28008
[333]	train-logloss:0.26446	eval-logloss:0.28003
[334]	train-logloss:0.26441	eval-logloss:0.28003
[335]	train-logloss:0.26437	eval-logloss:0.28007
[336]	train-logloss:0.26430	eval-logloss:0.28011
[337]	train-logloss:0.26427	eval-logloss:0.28014
[338]	train-logloss:0.26419	eval-logloss:0.28012
[339]	train-logloss:0.26412	eval-logloss:0.28010
[340]	train-logloss:0.26410	eval-logloss:0.28010
[341]	train-logloss:0.26403	eval-logloss:0.28012
[342]	train-logloss:0.26396	eval-logloss:0.28016
[343]	train-logloss:0.26389	eval-logloss:0.28012
[344]	train-logloss:0.26380	eval-logloss:0.28008
[345]	train-logloss:0.26373	eval-logloss:0.28013
[346]	train-logloss:0.26367	eval-logloss:0.28017
[347]	train-logloss:0.26362	eval-logloss:0.28018
[348]	train-logloss:0.26357	eval-logloss:0.28017
[349]	train-logloss:0.26356	eval-logloss:0.28018
[350]	train-logloss:

In [59]:
pred_probs = xgb_model.predict(dtest)
print('predict() 수행 결과값을 10개만 표시, 예측 확률값으로 표시됨')
print(np.round(pred_probs[:10], 3))

preds = [1 if x > 0.5 else 0 for x in pred_probs]

predict() 수행 결과값을 10개만 표시, 예측 확률값으로 표시됨
[0.505 0.374 1.    0.951 1.    1.    0.221 1.    1.    1.   ]


In [60]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy, precision, recall))

In [61]:
get_clf_eval(y_test, preds)

오차행렬
[[1930  469]
 [ 635 4507]]
정확도: 0.8536, 정밀도: 0.9057, 재현율: 0.8765


In [62]:
new = pd.read_csv('human_new.csv', encoding='cp949')
new.head()

,아이디,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,자본 이득,자본 손실,주당 시간,모국
0,H00001,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,0,0,40,United-States
1,H00002,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,0,0,50,United-States
2,H00003,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,0,0,40,United-States
3,H00004,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,7688,0,40,United-States
4,H00005,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,0,0,30,United-States


In [63]:
X_train.head()

,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,자본 이득,자본 손실,주당 시간,모국
23269,35,2,297485,9,13,0,3,4,4,0,0,40,38
18129,44,2,172032,8,11,2,0,0,4,7298,0,51,38
16236,27,2,472070,8,11,4,6,1,4,0,0,40,38
698,30,2,189620,9,13,4,9,3,4,0,0,40,30
19661,18,2,123714,1,7,4,5,1,2,0,0,40,38


In [64]:
from sklearn.impute import SimpleImputer
imputer_con = SimpleImputer(strategy="most_frequent")  
con = ['노동 계급', '학력', '혼인 상태', '직업', '관계', '인종', '모국']
imputer_con.fit(new[con])

SimpleImputer(strategy='most_frequent')

In [65]:
x = imputer_con.transform(new[con])
x

array([[' Private', ' 11th', ' Never-married', ..., ' Own-child',
        ' Black', ' United-States'],
       [' Private', ' HS-grad', ' Married-civ-spouse', ..., ' Husband',
        ' White', ' United-States'],
       [' Local-gov', ' Assoc-acdm', ' Married-civ-spouse', ...,
        ' Husband', ' White', ' United-States'],
       ...,
       [' Private', ' Masters', ' Never-married', ..., ' Not-in-family',
        ' White', ' United-States'],
       [' Private', ' Masters', ' Married-civ-spouse', ..., ' Husband',
        ' White', ' United-States'],
       [' Private', ' Masters', ' Married-civ-spouse', ..., ' Husband',
        ' White', ' United-States']], dtype=object)

In [66]:
new[con] = x
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   아이디     16281 non-null  object
 1   나이      16281 non-null  int64 
 2   노동 계급   16281 non-null  object
 3   fnlwgt  16281 non-null  int64 
 4   학력      16281 non-null  object
 5   교육 수    16281 non-null  int64 
 6   혼인 상태   16281 non-null  object
 7   직업      16281 non-null  object
 8   관계      16281 non-null  object
 9   인종      16281 non-null  object
 10  자본 이득   16281 non-null  int64 
 11  자본 손실   16281 non-null  int64 
 12  주당 시간   16281 non-null  int64 
 13  모국      16281 non-null  object
dtypes: int64(6), object(8)
memory usage: 1.7+ MB


In [67]:
new.isnull().sum()

아이디       0
나이        0
노동 계급     0
fnlwgt    0
학력        0
교육 수      0
혼인 상태     0
직업        0
관계        0
인종        0
자본 이득     0
자본 손실     0
주당 시간     0
모국        0
dtype: int64

In [68]:
new.shape

(16281, 14)

In [69]:
X_train.shape

(22621, 13)

In [70]:
new.isnull().sum()

아이디       0
나이        0
노동 계급     0
fnlwgt    0
학력        0
교육 수      0
혼인 상태     0
직업        0
관계        0
인종        0
자본 이득     0
자본 손실     0
주당 시간     0
모국        0
dtype: int64

In [71]:
obj1 = ['노동 계급', '학력', '혼인 상태', '직업', '관계', '인종', '모국']
new[obj1] = new[obj1].apply(lambda x: x.astype('category').cat.codes)
new.head()

,아이디,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,자본 이득,자본 손실,주당 시간,모국
0,H00001,25,3,226802,1,7,4,6,3,2,0,0,40,37
1,H00002,38,3,89814,11,9,2,4,0,4,0,0,50,37
2,H00003,28,1,336951,7,12,2,10,0,4,0,0,40,37
3,H00004,44,3,160323,15,10,2,6,0,2,7688,0,40,37
4,H00005,18,3,103497,15,10,4,9,3,4,0,0,30,37


In [72]:
ndf = new.copy()
ndf.head()

,아이디,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,자본 이득,자본 손실,주당 시간,모국
0,H00001,25,3,226802,1,7,4,6,3,2,0,0,40,37
1,H00002,38,3,89814,11,9,2,4,0,4,0,0,50,37
2,H00003,28,1,336951,7,12,2,10,0,4,0,0,40,37
3,H00004,44,3,160323,15,10,2,6,0,2,7688,0,40,37
4,H00005,18,3,103497,15,10,4,9,3,4,0,0,30,37


In [73]:
ndf['성별'] = tree.predict(ndf.loc[:,'나이':'모국'])
ndf.head()

,아이디,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,자본 이득,자본 손실,주당 시간,모국,성별
0,H00001,25,3,226802,1,7,4,6,3,2,0,0,40,37,1
1,H00002,38,3,89814,11,9,2,4,0,4,0,0,50,37,1
2,H00003,28,1,336951,7,12,2,10,0,4,0,0,40,37,1
3,H00004,44,3,160323,15,10,2,6,0,2,7688,0,40,37,1
4,H00005,18,3,103497,15,10,4,9,3,4,0,0,30,37,0


In [74]:
#df_tr.loc[df_tr['노동 계급'] == ' ?', '노동 계급'] = np.nan
#ndf.loc[ndf['성별']==1,'성별'] = 'Male'
#ndf.loc[ndf['성별']==0,'성별'] = 'Female'
#ndf.head()

In [75]:
dtest_ndf = xgb.DMatrix(data=ndf.loc[:,'나이':'모국'])

In [76]:
ndf['성별'] = xgb_model.predict(dtest_ndf)

In [77]:
ndf.head()

,아이디,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,자본 이득,자본 손실,주당 시간,모국,성별
0,H00001,25,3,226802,1,7,4,6,3,2,0,0,40,37,0.786478
1,H00002,38,3,89814,11,9,2,4,0,4,0,0,50,37,0.999972
2,H00003,28,1,336951,7,12,2,10,0,4,0,0,40,37,0.999950
3,H00004,44,3,160323,15,10,2,6,0,2,7688,0,40,37,0.999967
4,H00005,18,3,103497,15,10,4,9,3,4,0,0,30,37,0.458718


In [78]:
str = ['아이디','성별']
ndf[str].head()

,아이디,성별
0,H00001,0.786478
1,H00002,0.999972
2,H00003,0.999950
3,H00004,0.999967
4,H00005,0.458718


In [79]:
#ndf[str].to_csv('sampleSubmission_human.csv', encoding='cp949', index=False)

In [80]:
#movie_rename = movie.rename(index={0:'아바타'}, columns={'color':'색깔','director_name':'감독이름'})
ndf_rename = ndf.rename(columns={'아이디':'ID', '성별':'SEX'})
ndf_rename.head()

,ID,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,자본 이득,자본 손실,주당 시간,모국,SEX
0,H00001,25,3,226802,1,7,4,6,3,2,0,0,40,37,0.786478
1,H00002,38,3,89814,11,9,2,4,0,4,0,0,50,37,0.999972
2,H00003,28,1,336951,7,12,2,10,0,4,0,0,40,37,0.999950
3,H00004,44,3,160323,15,10,2,6,0,2,7688,0,40,37,0.999967
4,H00005,18,3,103497,15,10,4,9,3,4,0,0,30,37,0.458718


In [81]:
str = ['ID','SEX']
ndf_rename[str].head()

,ID,SEX
0,H00001,0.786478
1,H00002,0.999972
2,H00003,0.999950
3,H00004,0.999967
4,H00005,0.458718


In [84]:
ndf_rename[str].to_csv('submission_human4.csv', encoding='cp949', index=False)

In [85]:
ndf_rename.shape

(16281, 15)